In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../utils/")

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, MaxAbsScaler, RobustScaler

from GraphAPI import GraphCreator
from graph_helpers import *
from evaluations import *

%aimport GraphAPI
%aimport graph_helpers
%aimport evaluations

## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [2]:
# include_see_also=False used for validation below.
# in deployment, include_see_also should be set to True
gc = GraphCreator("Random forest", include_see_also=False) 
print("Number of Links to Search:", len(gc.next_links), "\n\n")
print(list(gc.primary_nodes.keys()), "\n\n")
print(gc.see_also_articles)

Number of Links to Search: 295 


['Regression analysis', 'Decision tree learning', 'Mode (statistics)', 'Adele Cutler', 'Tin Kam Ho', 'Bootstrap aggregating', 'Donald Geman', 'Leo Breiman', 'Random subspace method', 'Ensemble learning', 'Test set', 'Minitab', 'Overfitting', 'Statistical classification', 'Trademark'] 


['Boosting (machine learning)', 'Decision tree learning', 'Ensemble learning', 'Gradient boosting', 'Non-parametric statistics', 'Randomized algorithm']


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [3]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect nodes. 

In [4]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [5]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Deeplearning4j,Caffe (software),11
1,Caffe (software),Deeplearning4j,11
2,BigDL,Caffe (software),10
3,BigDL,Deeplearning4j,10
4,Trevor Hastie,Robert Tibshirani,7


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [6]:
features_df = gc.get_features_df(rank=False)

# Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [7]:
gc.rank_similarity()
gc.features_df.sort_values("similarity_rank", ascending=False).head(10)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link,similarity_rank
72,Random forest,289,1,162,127,1.000000,0.117804,0.000432,0.052576,0.0,0.0,1.000000,0,5.120505
434,Random subspace method,30,2,6,24,0.068807,0.007338,0.000016,0.001252,1.0,1.0,0.024390,1,2.947691
135,Gradient boosting,137,3,25,112,0.406780,0.015286,0.000063,0.006259,1.0,1.0,0.068571,0,2.854681
101,Bootstrap aggregating,230,2,130,100,0.588496,0.093819,0.000229,0.047568,1.0,1.0,0.678161,1,2.789087
60,Decision tree learning,326,2,172,154,0.494881,0.089342,0.000284,0.058209,1.0,1.0,0.590476,1,2.464255
166,Out-of-bag error,92,2,4,88,0.393365,0.004701,0.000013,0.001878,1.0,1.0,0.012195,0,2.114576
76,Boosting (machine learning),282,2,137,145,0.507407,0.089142,0.000220,0.050072,1.0,1.0,0.661111,0,1.872310
65,Ensemble learning,310,1,142,168,0.459732,0.089595,0.000238,0.050072,1.0,1.0,0.643243,1,1.786998
170,Logic learning machine,90,2,1,89,0.386792,0.002034,0.000004,0.000626,2.0,1.0,0.006173,0,1.464106
131,AdaBoost,146,2,30,116,0.354839,0.008144,0.000048,0.009389,2.0,1.0,0.037838,0,1.384002


# Scaling Features

We can easily scale our each of our features through the `scale_features_df` method. It will default to `Standard Scaler`, but we can specify alternate scalers in the `scaler` parameter.  

In [8]:
scaled_feature_df = gc.scale_features_df(scaler=MinMaxScaler, copy=True) # Makes a copy of the df
scaled_feature_df.sort_values("similarity_rank", ascending=False).reset_index().drop("index", axis=1)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link,similarity_rank
0,Random forest,0.013918,0.333333,0.007907,0.074794,1.000000,7.333117e-01,3.686650e-03,0.168337,0.0,0.0,1.000000,0.0,1.000000
1,Random subspace method,0.001402,0.666667,0.000293,0.014134,0.068807,4.567981e-02,1.091428e-04,0.004008,0.2,0.2,0.024390,1.0,0.575647
2,Gradient boosting,0.006573,1.000000,0.001220,0.065960,0.406780,9.515331e-02,5.157101e-04,0.020040,0.2,0.2,0.068571,0.0,0.557482
3,Bootstrap aggregating,0.011067,0.666667,0.006345,0.058893,0.588496,5.840071e-01,1.942120e-03,0.152305,0.2,0.2,0.678161,1.0,0.544671
4,Decision tree learning,0.015707,0.666667,0.008395,0.090695,0.494881,5.561370e-01,2.411374e-03,0.186373,0.2,0.2,0.590476,1.0,0.481231
5,Out-of-bag error,0.004398,0.666667,0.000195,0.051826,0.393365,2.926108e-02,8.030136e-05,0.006012,0.2,0.2,0.012195,0.0,0.412938
6,Boosting (machine learning),0.013580,0.666667,0.006687,0.085395,0.507407,5.548928e-01,1.859415e-03,0.160321,0.2,0.2,0.661111,0.0,0.365624
7,Ensemble learning,0.014933,0.333333,0.006931,0.098940,0.459732,5.577153e-01,2.015787e-03,0.160321,0.2,0.2,0.643243,1.0,0.348962
8,Logic learning machine,0.004301,0.666667,0.000049,0.052415,0.386792,1.266157e-02,3.247289e-06,0.002004,0.4,0.2,0.006173,0.0,0.285901
9,AdaBoost,0.007008,0.666667,0.001464,0.068316,0.354839,5.069294e-02,3.813098e-04,0.030060,0.4,0.2,0.037838,0.0,0.270256


___
# Validation

Here, we _validate_ our results. For many articles, we already have some user defined links that are highly related to the present article. These are found in the **See Also** section of several Wikipedia articles (some pages do not have them). These are not ordered in importance in any meaningful way, and there are no rating scores.

The intuition in this validation is as follows: 

> _Given that we know some articles are highly related from user input, if the recommendations provided by this system are valid, we would expect to see those **See Also** links ranked relatively high on our list._ 

_Note: This validation is not meant as **confirmation** or **evaluation** of the results. It only provides us one way of telling if the results we are seeing are reasonably valid. It is important to note that we cannot compare these results across two different articles, as those would be two entirely different network structures, likely with different human labeled links._  


In [9]:
evaluate_metrics(scaled_feature_df, 
                 on=["similarity_rank", "centrality", "adjusted_reciprocity", "page_rank", "shortest_path_length_from_entry", "jaccard_similarity"], 
                 targets=gc.see_also_articles)

Metric Score,score,max score possible,difference,total targets,% targets in top 1%,% targets in top 5%,% targets in top 10%,% targets in top 20%
similarity_rank,0.989423,0.99994,0.010517,5.0,0.8,1.0,1.0,1.0
centrality,0.995078,0.99994,0.004862,5.0,1.0,1.0,1.0,1.0
adjusted_reciprocity,0.995690,0.99994,0.004250,5.0,1.0,1.0,1.0,1.0
page_rank,0.991272,0.99994,0.008668,5.0,1.0,1.0,1.0,1.0
shortest_path_length_from_entry,0.000192,0.99994,0.999748,5.0,0.0,0.0,0.0,0.0
jaccard_similarity,0.970526,0.99994,0.029414,5.0,0.8,1.0,1.0,1.0


The chart generated above compares different ranking metrics (left index) for a given article. The most important column, `score`, provides a fast way for us to compare these different metrics. 

For example, if we see a _score_ of 0.98 for a given ranking metric, The following statement would be true:

> All of the human labeled **See Also** links are present within the top 98% of our recommendations. 

Since the human labeled links comprise a range, it is not possible to get a score of 100%. The `max score possible` column indicates the score that would be achieved if all the human labeled _See Also_ links appeared at the top of our recommendations without any other links intervening.   

The `difference` column is an alternative way of looking at the score. If we had a 0.02 in this column, we could say:

> All the human labeled **See Also** links are contained within the top 2% of our recommendations. 

`Total targets` is the number of human labeled _See Also_ links. 

Because it is possible that different metrics could have similar scores, we want a way to break down the dispersion of the known related links to see if one metric does perform better than another. The trailing four columns provide us with a course way of measuring this dispersion. 

Each of these columns indicates the percentage of human labeled _See Also_ links captured within a given percentage of the top of our recommendations. For example, if we see a 0.92 in the `% targets in the top 1%` columns, we could say:

> 92% of the human labeled **See Also** links appear in the top 1% of our recommendations. 

The value of these columns is a follows - If two ranking metrics have similar scores, we _might_ consider the better performing one to be the one in which the majority of the human labeled links are higher in our recommendation list. 

___

In [10]:
gc.features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83300 entries, 0 to 83299
Data columns (total 14 columns):
node                                 83300 non-null object
degree                               83300 non-null int64
category_matches_with_source         83300 non-null int64
in_edges                             83300 non-null int64
out_edges                            83300 non-null int64
shared_neighbors_with_entry_score    83300 non-null float64
centrality                           83300 non-null float64
page_rank                            83300 non-null float64
adjusted_reciprocity                 83300 non-null float64
shortest_path_length_from_entry      83300 non-null float64
shortest_path_length_to_entry        83300 non-null float64
jaccard_similarity                   83300 non-null float64
primary_link                         83300 non-null int64
similarity_rank                      83300 non-null float64
dtypes: float64(8), int64(5), object(1)
memory usage: 9.5+ MB


In [ ]:
len(gc.graph.nodes)